In [1]:
import sys
sys.version_info

sys.version_info(major=3, minor=5, micro=2, releaselevel='final', serial=0)

In [2]:
import socket
print(socket.gethostname())

1080


In [19]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import urllib.request

import tensorflow as tf
import numpy as np
tf.logging.set_verbosity(tf.logging.INFO)

IRIS_TRAINING = "iris_training.csv"
IRIS_TRAINING_URL = "http://download.tensorflow.org/data/iris_training.csv"

IRIS_TEST = "iris_test.csv"
IRIS_TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

In [4]:
if not os.path.exists(IRIS_TRAINING):
  raw = urllib.request.urlopen(IRIS_TRAINING_URL).read()
  with open(IRIS_TRAINING,'w') as f:
    f.write(raw.decode('utf-8'))

if not os.path.exists(IRIS_TEST):
  raw = urllib.request.urlopen(IRIS_TEST_URL).read()
  with open(IRIS_TEST,'w') as f:
    f.write(raw.decode('utf-8'))

In [5]:
# Load datasets.
training_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename=IRIS_TRAINING,
    target_dtype=np.int,
    features_dtype=np.float32)
test_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename=IRIS_TEST,
    target_dtype=np.int,
    features_dtype=np.float32)

In [6]:
#training_set[0]
##training_set??

In [43]:
# Specify that all features have real-value data
feature_columns = [tf.contrib.layers.real_valued_column("", dimension=4)]

validation_metrics = {
    "accuracy":
        tf.contrib.learn.MetricSpec(
            metric_fn=tf.contrib.metrics.streaming_accuracy,
            prediction_key=tf.contrib.learn.PredictionKey.CLASSES),
    "precision":
        tf.contrib.learn.MetricSpec(
            metric_fn=tf.contrib.metrics.streaming_precision,
            prediction_key=tf.contrib.learn.PredictionKey.CLASSES),
    "recall":
        tf.contrib.learn.MetricSpec(
            metric_fn=tf.contrib.metrics.streaming_recall,
            prediction_key=tf.contrib.learn.PredictionKey.CLASSES)
}

validation_monitor = tf.contrib.learn.monitors.ValidationMonitor(
    test_set.data,
    test_set.target,
    every_n_steps=50,
    metrics=validation_metrics,
    early_stopping_metric="loss",
    early_stopping_metric_minimize=True,
    early_stopping_rounds=200)
# Build 3 layer DNN with 10, 20, 10 units respectively.
classifier = tf.contrib.learn.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[10, 20, 10],
    n_classes=3,
    model_dir="/tmp/iris_model",
    config=tf.contrib.learn.RunConfig(save_checkpoints_secs=1))

Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.
INFO:tensorflow:Using config: {'_num_worker_replicas': 0, '_task_id': 0, '_save_checkpoints_secs': 1, '_task_type': None, '_evaluation_master': '', '_model_dir': '/tmp/iris_model', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7efca232b0b8>, '_num_ps_replicas': 0, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_is_chief': True, '_keep_checkpoint_max': 5, '_master': '', '_session_config': None, '_keep_checkpoint_every_n_hours': 10000, '_environment': 'local', '_save_checkpoints_steps': None, '_save_summary_steps': 100}


In [44]:
# Define the training inputs
def get_train_inputs():
  x = tf.constant(training_set.data)
  y = tf.constant(training_set.target)
  return x, y

In [45]:
classifier.fit(input_fn=get_train_inputs,steps=2000,monitors=[validation_monitor])
# Fit model.
#classifier.fit(input_fn=get_train_inputs, steps=2000)

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/iris_model/model.ckpt.
INFO:tensorflow:step = 1, loss = 1.12063
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will onl

INFO:tensorflow:Finished evaluation at 2017-06-23-03:57:06
INFO:tensorflow:Saving dict for global step 1164: accuracy = 0.966667, global_step = 1164, loss = 0.0518424, precision = 1.0, recall = 1.0
INFO:tensorflow:Validation (step 1200): global_step = 1164, precision = 1.0, loss = 0.0518424, recall = 1.0, accuracy = 0.966667
INFO:tensorflow:global_step/sec: 226.384
INFO:tensorflow:step = 1201, loss = 0.0492553 (0.442 sec)
INFO:tensorflow:global_step/sec: 985.092
INFO:tensorflow:step = 1301, loss = 0.0482659 (0.102 sec)
INFO:tensorflow:global_step/sec: 1006.7
INFO:tensorflow:step = 1401, loss = 0.0474142 (0.100 sec)
INFO:tensorflow:global_step/sec: 968.393
INFO:tensorflow:step = 1501, loss = 0.0465613 (0.103 sec)
INFO:tensorflow:global_step/sec: 958.288
INFO:tensorflow:step = 1601, loss = 0.0458181 (0.104 sec)
INFO:tensorflow:global_step/sec: 993.043
INFO:tensorflow:step = 1701, loss = 0.045049 (0.101 sec)
INFO:tensorflow:global_step/sec: 973.043
INFO:tensorflow:step = 1801, loss = 0.04

DNNClassifier(params={'embedding_lr_multipliers': None, 'hidden_units': [10, 20, 10], 'input_layer_min_slice_size': None, 'head': <tensorflow.contrib.learn.python.learn.estimators.head._MultiClassHead object at 0x7efce4bd6668>, 'dropout': None, 'optimizer': None, 'feature_columns': (_RealValuedColumn(column_name='', dimension=4, default_value=None, dtype=tf.float32, normalizer=None),), 'gradient_clip_norm': None, 'activation_fn': <function relu at 0x7efca7aa2950>})

In [10]:
# Define the test inputs
def get_test_inputs():
    x = tf.constant(test_set.data)
    y = tf.constant(test_set.target)
    return x, y


In [11]:
# Evaluate accuracy.
accuracy_score = classifier.evaluate(input_fn=get_test_inputs,
                                     steps=1)["accuracy"]

print("\nTest Accuracy: {0:f}\n".format(accuracy_score))


Test Accuracy: 0.966667



In [12]:
# Classify two new flower samples.
def new_samples():
  return np.array(
    [[6.4, 3.2, 4.5, 1.5],
     [5.8, 3.1, 5.0, 1.7]], dtype=np.float32)

predictions = list(classifier.predict(input_fn=new_samples))

print(
    "New Samples, Class Predictions:    {}\n"
    .format(predictions))

New Samples, Class Predictions:    [1, 2]

